In [1]:
import pandas as pd 
BASE_DISC_TRAIN_PATH = "/home/perm/sars_escape_netv2/data/disc/train/sarx"
DISCRIMINATOR_LEARNING_MODEL_PATH = "/home/perm/sars_escape_netv2/model/sarsx_disc"
non_sig_path = BASE_DISC_TRAIN_PATH+"/non-significantx.csv"
sig_path= BASE_DISC_TRAIN_PATH+"/significantx.csv"
df1 = pd.read_csv(sig_path)
sig_seqs = list(df1['window_seqs'])

df2 = pd.read_csv(non_sig_path)
non_sig_seqs = list(df2['window_seqs'])
print(f'Sig seq len: {len(sig_seqs)} , Non sig: {len(non_sig_seqs)} type: {type(non_sig_seqs)} ')
# print("Sig sample record: ",sig_seqs[0:2] )
# print("Non Sig sample record: ",non_sig_seqs[0:2] )




Sig seq len: 230997 , Non sig: 4381 type: <class 'list'> 
Sig sample record:  ['GVEGFNCYFPLQSYGFQPTN', 'KVGGNYNYRYRLFRKSNLKP']
Non Sig sample record:  ['PIKDFGGFNFSQILPDPSKP', 'LFQCYLVPCYSLGPMVLRGL']


In [17]:
import numpy as np 
rand_sig_indexes  = np.random.randint(0, len(sig_seqs), size=len(non_sig_seqs))
print(f'len of rand_array : {len(rand_sig_indexes)} , first 5 samples: {rand_sig_indexes[0:5]}')
# sig_seqs[rand_sig_indexes]
sig_samples  = np.take(sig_seqs, rand_sig_indexes)
print(f'Length of sig samples: {len(sig_samples)} , samples : {sig_samples[0:2]}')

len of rand_array : 4381 , first 5 samples: [ 17196 110865 130776 215308 213855]
Length of sig samples: 4381 , samples : ['YSFRPTYGVGHQPYRVVVLS' 'EIYQAGSTPCNGVGGFNCYF']


In [ ]:
import EscapeNet2Executer as ESC


In [20]:
def featurize_sarx(sequences):
    fz_model  = ESC.load_fz_model()
    X_cat, lengths  = ESC.featurize_seqs(sequences)
    y_embed_output = fz_model.transform(X_cat, lengths)
    print("Shape of output previously: ", y_embed_output.shape) 
    #Reducing the feature taking the avarage from middle axis
    y_embed_output = np.average(y_embed_output, axis=1)
    return y_embed_output

In [ ]:
sig_features  = featurize_sarx(sig_samples)
non_sig_features = featurize_sarx(non_sig_seqs)

In [ ]:
non_sig_train  = get_features()
non_sig_test  = get_features(BASE_DISC_TRAIN_PATH+"/non_sig_combined_windows_test.csv")
sig_train_orig  = get_features(BASE_DISC_TRAIN_PATH+"/significantx.csv")



# sig_train_escape_aug = get_features_gan(GAN_AUG_SEQS)

#Combine features from augmentation with sig_train_orig
# sig_train_orig = np.concatenate( (sig_train_orig, sig_train_escape_aug), axis=0 )
#Shuffle happens in place : ie orginal array directly modified
np.random.shuffle(sig_train_orig)

sig_test =  get_features(BASE_DISC_TRAIN_PATH+"/sig_combined_windows_test.csv")

train_ds, val_ds = get_balanced_datasets_over_sampling(non_sig_train, non_sig_test, sig_train_orig, sig_test)

model  = get_dense_model()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5) #  
model.fit(train_ds,  epochs=NO_OF_EPOCHS_TO_TRAIN_DISC_NET, validation_data=val_ds,  callbacks= [callback], verbose=1) #verbose 1 = Progress bar

model.save(DISCRIMINATOR_LEARNING_MODEL_PATH)
print("**-------Discriminator Network sucessfully trained and saved to path-----***",DISCRIMINATOR_LEARNING_MODEL_PATH)
